# DATA20001 Deep Learning - Group Project
## Text project

**Due Thursday, May 22, before 23:59.**

The task is to learn to assign the correct labels to news articles.  The corpus contains ~850K articles from Reuters.  The test set is about 10% of the articles. The data is unextracted in XML files.

We're only giving you the code for downloading the data, and how to save the final model. The rest you'll have to do yourselves.

Some comments and hints particular to the project:

- One document may belong to many classes in this problem, i.e., it's a multi-label classification problem. In fact there are documents that don't belong to any class, and you should also be able to handle these correctly. Pay careful attention to how you design the outputs of the network (e.g., what activation to use) and what loss function should be used.
- You may use word-embeddings to get better results. For example, you were already using a smaller version of the GloVE  embeddings in exercise 4. Do note that these embeddings take a lot of memory. 
- In the exercises we used e.g., `torchvision.datasets.MNIST` to handle the loading of the data in suitable batches. Here, you need to handle the dataloading yourself.  The easiest way is probably to create a custom `Dataset`. [See for example here for a tutorial](https://github.com/utkuozbulak/pytorch-custom-dataset-examples).

In [16]:
import pickle
import warnings
from multiprocessing import cpu_count

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import tensorflow as tf
from gensim.models import Word2Vec
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Dense, Dropout, Activation, Embedding, Conv1D, \
        GlobalMaxPooling1D, SpatialDropout1D, LSTM, GRU, Flatten, MaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import data
import preprocessing

seed = 42

The above command downloads and extracts the data files into the `train` subdirectory.

The files can be found in `train/`, and are named as `19970405.zip`, etc. You will have to manage the content of these zips to get the data. There is a readme which has links to further descriptions on the data.

The class labels, or topics, can be found in the readme file called `train/codes.zip`.  The zip contains a file called "topic_codes.txt".  This file contains the special codes for the topics (about 130 of them), and the explanation - what each code means.

The XML document files contain the article's headline, the main body text, and the list of topic labels assigned to each article.  You will have to extract the topics of each article from the XML.  For example: 
&lt;code code="C18"&gt; refers to the topic "OWNERSHIP CHANGES" (like a corporate buyout).

You should pre-process the XML to extract the words from the article: the &lt;headline&gt; element and the &lt;text&gt;.  You should not need any other parts of the article.

## Extracting the data

In [17]:
# data.extract_data(extraction_dir="train", data_dir="data", data_zip_name="reuters-training-corpus.zip")

try:
    df = pd.read_pickle("train/data.pkl")
except FileNotFoundError:
    df = data.get_docs_labels("train/REUTERS_CORPUS_2")
    df.to_pickle("train/data.pkl")

docs = df["doc"].values
labels = np.array(df["labels"].tolist())
n_labels = len(data.CODEMAP)

print(docs.shape)
print(labels.shape)
print(docs[-2])
print(labels[-2])

(299773,)
(299773, 126)
Typhoon Winnie kills 25 in Taiwan. A typhoon that packed high winds and torrential rain killed 25 people in Taiwan on Monday and Tuesday, with landslides bringing down buildings and floodwaters turning streets into rivers, officials said on Tuesday. The death toll has risen to 25, one missing, 16 seriously injured and 62 slightly hurt, the government's anti-typhoon centre said in a statement. Three houses totally collapsed and 37 partly collapsed, it said. State television showed several five-storey buildings in eastern Taipei that had sunk two stories into the ground. The Central Weather Bureau said late on Monday the danger had passed as Typhoon Winnie headed towards mainland China. Heavy torrential rain and strong winds triggered landslides in Taipei, destroying or damaging buildings and blocking traffic. "The whole scene looks as if it has gone through an explosion," a state television reporter in the city said. Local authorities mobilised hundreds of rescue

## Preprocessing the data

In [18]:
path_to_prepocessed_docs = "train/preprocessed_docs.pkl"
try:
    with open(path_to_prepocessed_docs, "rb") as f:
        preprocessed_docs = pickle.load(f)
except FileNotFoundError:
    preprocessed_docs = preprocessing.preprocess_corpus(docs)
    with open(path_to_prepocessed_docs, "wb") as f:
        pickle.dump(preprocessed_docs, f)

preprocessed_docs = [s.split() for s in preprocessed_docs]

print(" ".join(preprocessed_docs[-2]))

typhoon winnie kill -num- taiwan typhoon pack high wind torrential rain kill -num- people taiwan monday tuesday landslide bring building floodwater turn street river official say tuesday death toll rise -num- miss -num- seriously injure -num- slightly hurt government anti typhoon centre say statement house totally collapse -num- partly collapse say state television show storey building eastern taipei sink story ground central weather bureau say late monday danger pass typhoon winnie head mainland china heavy torrential rain strong wind trigger landslide taipei destroy damaging building block traffic scene look go explosion state television reporter city say local authority mobilise hundred rescue worker soldier help evacuate resident people remain trap house state medium say -num- people bury landslide hit house north taipei woman survive torrential rain swamp house low lie area capital road flood score vehicle submerge typhoon pack maximum sustained wind -num- km hour -num- mile hour 

In [19]:
path_to_tokenizer = "train/tokenizer.pkl"
n_sequence = max([len(doc) for doc in preprocessed_docs])
n_vocabulary = 30000

try:
    with open(path_to_tokenizer, "rb") as f:
        tokenizer = pickle.load(f)
except FileNotFoundError:
    tokenizer = Tokenizer(num_words=n_vocabulary, filters="")
    # tokenizer = Tokenizer(filters="")
    tokenizer.fit_on_texts(preprocessed_docs)
    with open(path_to_tokenizer, "wb") as f:
        pickle.dump(tokenizer, f)

word_idx = tokenizer.word_index
# n_vocabulary = len(word_idx)

# doc_matrix = tokenizer.texts_to_matrix(preprocessed_docs).astype(np.int32)
sequences = tokenizer.texts_to_sequences(preprocessed_docs)
sequences = pad_sequences(sequences, maxlen=n_sequence, padding="post")

print(n_sequence)
print(n_vocabulary)
# print(doc_matrix.shape)
print(sequences.shape)

# print(doc_matrix[1])
print(sequences[1])

7816
30000
(299773, 7816)
[   36  1885 19824 ...     0     0     0]


In [20]:
n_embedding = 200

try:
    w2v_model = Word2Vec.load("train/w2v.model")
except FileNotFoundError:
    w2v_model = Word2Vec(sentences=preprocessed_docs,
                         size=n_embedding, 
                         window=5,
                         workers=cpu_count(),
                         min_count=1)
    w2v_model.save("train/w2v.model")

n_vocabulary_w2v = len(list(w2v_model.wv.vocab))
print(n_vocabulary_w2v)

306403


In [21]:
embedding_matrix = np.zeros((n_vocabulary + 1, n_embedding))
for token, i in word_idx.items():
    if i > n_vocabulary:
        continue
    if token in w2v_model:
        embedding_matrix[i] = w2v_model[token]
    else:
        embedding_matrix[i] = np.zeros(n_embedding)

print(embedding_matrix.shape)

(30001, 200)


In [34]:
model = Sequential()

model.add(Embedding(
    n_vocabulary + 1,
    n_embedding,
    embeddings_initializer=Constant(embedding_matrix),
    input_length=n_sequence,
    trainable=False
))
# model.add(Embedding(n_vocabulary + 1, n_embedding, input_length=n_sequence))

# model.add(GRU(32, dropout=.2))

model.add(Conv1D(100, 4, activation="relu"))
model.add(MaxPooling1D(pool_size=3))
model.add(Conv1D(100, 2, activation="relu"))
model.add(Dropout(.5))
model.add(Flatten())
model.add(Dense(300, activation="relu"))

model.add(Dense(n_labels, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 7816, 200)         6000200   
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 7813, 100)         80100     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 2604, 100)         0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 2603, 100)         20100     
_________________________________________________________________
dropout_3 (Dropout)          (None, 2603, 100)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 260300)            0         
_________________________________________________________________
dense_9 (Dense)              (None, 300)              

In [35]:
batch_size = 256
n = 20000

x_train, x_test, y_train, y_test = train_test_split(sequences,
                                                    labels,
                                                    train_size=n,
                                                    test_size=n,
                                                    random_state=seed)

es = EarlyStopping(patience=3, verbose=1, restore_best_weights=True)
history = model.fit(x_train,
                    y_train,
                    batch_size=batch_size,
                    epochs=100,
                    verbose=1,
                    validation_split=.1,
                    callbacks=[es])

y_pred = model.predict(x_test, batch_size=batch_size, verbose=1) > .2

print(f"test accuracy: {accuracy_score(y_test, y_pred)}")
print(f"test f1-score (micro): {f1_score(y_test, y_pred, average='micro')}")
print(f"test f1-score (macro): {f1_score(y_test, y_pred, average='macro')}")

Train on 18000 samples, validate on 2000 samples
Epoch 1/100
18000/18000 [==============================] - 25s 1ms/sample - loss: 0.0825 - accuracy: 0.9719 - val_loss: 0.0414 - val_accuracy: 0.9868
Epoch 2/100
18000/18000 [==============================] - 23s 1ms/sample - loss: 0.0365 - accuracy: 0.9879 - val_loss: 0.0332 - val_accuracy: 0.9889
Epoch 3/100
18000/18000 [==============================] - 23s 1ms/sample - loss: 0.0285 - accuracy: 0.9902 - val_loss: 0.0304 - val_accuracy: 0.9900
Epoch 4/100
18000/18000 [==============================] - 24s 1ms/sample - loss: 0.0236 - accuracy: 0.9917 - val_loss: 0.0288 - val_accuracy: 0.9904
Epoch 5/100
18000/18000 [==============================] - 23s 1ms/sample - loss: 0.0200 - accuracy: 0.9928 - val_loss: 0.0290 - val_accuracy: 0.9907
Epoch 6/100
18000/18000 [==============================] - 23s 1ms/sample - loss: 0.0175 - accuracy: 0.9937 - val_loss: 0.0293 - val_accuracy: 0.9909
Epoch 7/100
18000/18000 [==========================

## Save your model

It might be useful to save your model if you want to continue your work later, or use it for inference later.

In [24]:
# torch.save(model.state_dict(), 'model.pkl')

The model file should now be visible in the "Home" screen of the jupyter notebooks interface.  There you should be able to select it and press "download".

## Download test set

The testset will be made available during the last week before the deadline and can be downloaded in the same way as the training set.

## Predict for test set

You will be asked to return your predictions a separate test set.  These should be returned as a matrix with one row for each test article.  Each row contains a binary prediction for each label, 1 if it's present in the image, and 0 if not. The order of the labels is the order of the label (topic) codes.

An example row could like like this if your system predicts the presense of the second and fourth topic:

    0 1 0 1 0 0 0 0 0 0 0 0 0 0 ...
    
If you have the matrix prepared in `y` you can use the following function to save it to a text file.

In [25]:
# np.savetxt('results.txt', y, fmt='%d')